In [1]:
import torch

# Natural Language Processing en PyTorch

El objetivo ahora es describir con un ejemplo el ciclo de trabajo que se usa para entrenar un algoritmo de NLP. En este caso se hará **clasificación de texto sobre el corpus AG NEWS**: se desea predecir, a partir de una frase perteneciente al corpus, el tipo de noticia (Internacional, Deportes, Negocios o Ciencia/Tecnología) a la que pertenece. La idea no es implementar un modelo muy complicado, sino estudiar todo el proceso de programación al rededor del mismo. Esto incluye:

1. Definir las muestras a partir de del corpus de texto en una forma eficiente. En este caso se dispone del corpus *AG NEWS*, conformado por una colección de noticias pertenecientes a distintas categorías. De cada noticia se conoce a qué categoría pertenece. Con esto, se definen los conjuntos de entrenamiento, validación y testeo. 

2. Crear un objeto que permita manejar las muestras en un formato "utilizable". Este objeto se conoce como Dataloader y hay uno por cada conjunto de muestras. 

3. Definir un modelo simple y el grafo computacional que permita ejecutar el algoritmo de *backpropagation* para encontrar los parámetros del modelo.

4. Crear las funciones para visualizar que el entrenamiento se está realizando con éxito. Esto generalmente incluye el loop principal del algoritmo y el cálculo de la cantidad de aciertos en el conjunto de validación.


Resumen del ciclo de trabajo:

* Nuestro dataset es conjunto de muestras, cada una de ellas conformada por una secuencia de texto y un label. Además, se dispone de un vocabulario, de manera que todas las palabras de la secuencia de texto de la muestra se pueden mapear a un índice en ese vocabulario (incluído los caracteres especiales para denotar "fin de párrafo", "palabra desconocida", "padding token", etc.). Todo esto se va a definir en una subclase de la clase `torch.utils.data.Dataset`, en donde se sobreescriben los métodos `__len__` y `__getitem__`.

* Una vez definido el dataset, se obtiene el dataloader, el cual es una subclase de la clase `torch.utils.data.DataLoader` y recibe como argumentos una función para muestrear, el tamaño del batch y la instancia de `torch.utils.data.Dataset` definida anteriormente.

* Por otro lado, se define el objeto que representa al modelo, lo cual se hace creando una subclase de `torch.nn.Module`.

* Finalmente se hace el entrenamiento y se verifican los resultados en el conjunto de validación. 

## Creación de las muestras

Nuestro dataset es conjunto de muestras, cada una de ellas conformada por una secuencia de palabras o *tokens*, que conforman una frase de una noticia del corpus, y un label que indica la categoría a la que pertenece esa secuencia de tokens. 

    sample1 = ( ['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.', 'wall st', 'st .', '. bears', 'bears claw', 'claw back', 'back into', 'into the', 'the black', 'black (', '( reuters', 'reuters )', ') reuters', 'reuters -', '- short-sellers', 'short-sellers ,', ', wall', 'wall street', "street '", "' s", 's dwindling\\band', 'dwindling\\band of', 'of ultra-cynics', 'ultra-cynics ,', ', are', 'are seeing', 'seeing green', 'green again', 'again .'], 'Business' )
    
    sample2 = ( ['european', 'union', 'extends', 'microsoft-time', 'warner', 'review', 'brussels', ',', 'belgium', '(', 'ap', ')', '--', 'european', 'antitrust', 'regulators', 'said', 'monday', 'they', 'have', 'extended', 'their', 'review', 'of', 'a', 'deal', 'between', 'microsoft', 'corp', '.', '(', 'msft', ')', 'and', 'time', 'warner', 'inc', '.', '.', '.', 'european union', 'union extends', 'extends microsoft-time', 'microsoft-time warner', 'warner review', 'review brussels', 'brussels ,', ', belgium', 'belgium (', '( ap', 'ap )', ') --', '-- european', 'european antitrust', 'antitrust regulators', 'regulators said', 'said monday', 'monday they', 'they have', 'have extended', 'extended their', 'their review', 'review of', 'of a', 'a deal', 'deal between', 'between microsoft', 'microsoft corp', 'corp .', '. (', '( msft', 'msft )', ') and', 'and time', 'time warner', 'warner inc', 'inc .', '. .', '. .'], 'Sci/Tec')
    
    ...
    
    sampleN = ( ['patriots', 'sign', 'top', 'pick', 'watson', '(', 'reuters', ')', 'reuters', '-', 'the', 'new', 'england', 'patriots\\monday', 'announced', 'the', 'signing', 'of', 'first-round', 'draft', 'pick', 'benjamin\\watson', '.', 'as', 'per', 'team', 'policy', ',', 'terms', 'of', 'the', 'tight', 'end', "'", 's', 'deal', 'were\\not', 'released', '.', 'patriots sign', 'sign top', 'top pick', 'pick watson', 'watson (', '( reuters', 'reuters )', ') reuters', 'reuters -', '- the', 'the new', 'new england', 'england patriots\\monday', 'patriots\\monday announced', 'announced the', 'the signing', 'signing of', 'of first-round', 'first-round draft', 'draft pick', 'pick benjamin\\watson', 'benjamin\\watson .', '. as', 'as per', 'per team', 'team policy', 'policy ,', ', terms', 'terms of', 'of the', 'the tight', 'tight end', "end '", "' s", 's deal', 'deal were\\not', 'were\\not released', 'released .'], 'Sports')

Para hacer más eficiente la definición de las muestras se define el **vocabulario**, que contiene todas las palabras que aparecen en el corpus. De esta forma, la secuencia de palabras va a pasar a componerse del índice del vocabulario de cada una de estas palabras, y la categoría, a ser el índice en la lista de categorías `['World', 'Sports', 'Business', 'Sci/Tec']`.

    sample1 = (tensor([    572,     564,       2,    2326,   49106,     150,      88,       3,
                          1143,      14,      32,      15,      32,      16,  443749,       4,
                           572,     499,      17,      10,  741769,       7,  468770,       4,
                            52,    7019,    1050,     442,       2,   14341,     673,  141447,
                        326092,   55044,    7887,     411,    9870,  628642,      43,      44,
                           144,     145,  299709,  443750,   51274,     703,   14312,      23,
                       1111134,  741770,  411508,  468771,    3779,   86384,  135944,  371666,
                          4052], 2)

    sample2 = (tensor([   227,    377,   4085,  81800,   1910,   1790,   3038,      4,   6645,
                           14,     36,     15,     67,    227,   2631,   1384,     31,     74,
                           90,     49,   2393,     50,   1790,      7,      6,    164,    312,
                          101,    115,      2,     14,   4406,     15,      9,    134,   1910,
                           85,      2,      2,      2,   1068, 469452, 358665,  81801, 474085,
                       433611,  18201,  16373,  62492,     62,     63,    584,  71591,  44508,
                        26665,  25055,    979,  19427,   1845,  52443,  17357, 165870,  10476,
                          107,   1187,  23580,  51777,    981,    126,    384,  13942,  12119,
                         2404,  16413,   3225,   9773,     89,     37,     37]), 3 )

    ...

    sampleN = (tensor([   1968,    1025,     181,    3153,   22347,      14,      32,      15,
                            32,      16,       3,      27,     430, 1029946,     184,       3,
                          4162,       7,    7000,    3061,    3153,  622440,       2,      21,
                           677,     148,    1247,       4,    2358,       7,       3,    3389,
                           208,      17,      10,     164, 1283369,     466,       2,  418788,
                        205275,   84380,  420603,  474767,      43,      44,     144,     145,
                           119,     324,    2872,  753763, 1029947,    3056,   38172,   30591,
                        158144,   49428,   26084, 1037969,  622441,    3782,   54761,   75502,
                         70592,   15745,  170329,    7607,      29,   84113,   18112,  355484,
                            23,   32659,  713718, 1283370,   23833]), 1 )